In [1]:
import requests
from bs4 import BeautifulSoup 
import lxml
import csv
from datetime import datetime


#### Taking day date from user

In [6]:
# Accept input from user
date_string = input("Enter a date in the format mm/dd/yyyy: ")
print("You entered: ", date_string)

from datetime import datetime

# Convert date string to datetime object
date_object = datetime.strptime(date_string, '%m/%d/%Y').date()
print("After changing it to data: ", date_object)

# Convert datetime object back to string with new format
new_date_string = datetime.strftime(date_object, '%m-%d-%Y')
print("After converted back to string on the format %m-%d-%Y: ", new_date_string)


Enter a date in the format mm/dd/yyyy: 12/30/2022
You entered:  12/30/2022
After changing it to data:  2022-12-30
After converted back to string on the format %m-%d-%Y:  12-30-2022


In [7]:
# Requesting the webpage and adding the date to it
page = requests.get(f"https://www.yallakora.com/match-center?date={date_string}")
website_URL = "https://www.yallakora.com"

# Checking the response of the getting the page, if respose is 200; this means we got it fine
page

<Response [200]>

### Building function that extract match details through segment of the webpage and store it in csv file

In [35]:
def get_match_info(championships):
        # Looping through the championships
        #for championship in championships:
        
        match_details = []
        
        #championship_title = championships[1].find("h2").text.strip()
        championship_title = championships[1].find("a",{"class":"tourTitle"}).find_next("h2").text.strip()
        team_a = championships[1].find("div",{"class":"allData"}
                                  ).find_next("div", {"class":"teamsData"}
                                             ).find_next("div", {"class":"teams teamA"}
                                                        ).find_next("p").text
        team_b = championships[1].find("div",{"class":"allData"}
                                  ).find_next("div", {"class":"teamsData"}
                                             ).find_next("div", {"class":"teams teamB"}
                                                        ).find_next("p").text
        time = championships[1].find("div",{"class":"allData"}
                                  ).find_next("div", {"class":"teamsData"}
                                             ).find_next("div", {"class":"MResult"}
                                                        ).find_next("span"
                                                                   ).find_next("span"
                                                                              ).find_next("span"
                                                                                         ).find_next("span").text
        team_a_score = championships[1].find("div",{"class":"allData"}
                                  ).find_next("div", {"class":"teamsData"}
                                             ).find_next("div", {"class":"MResult"}
                                                        ).find_next("span").text
        
        team_b_score = championships[1].find("div",{"class":"allData"}
                                  ).find_next("div", {"class":"teamsData"}
                                             ).find_next("div", {"class":"MResult"}
                                                        ).find_next("span"
                                                                   ).find_next("span"
                                                                              ).find_next("span").text
        
        match_status = championships[1].find("div",{"class":"matchStatus"}).find_next('span').text
        
        
        chanel_streaming = championships[1].find("div",{"class":"channel icon-channel"}).text
        
        match_detail_link = website_URL + championships[1].find("div",{"class":"leftCol"}
                                                               ).find_next('a')['href']
        
        match_week_number = championships[1].find("div",{"class":"date"}).text

        
        # Storing the values in dictionary
        match = {'Date':date_object, 'Championship_Title':championship_title, 'Team_A':team_a, 'Team_B':team_b,
                 'Time':time, 'Team_A_Score':team_a_score, 'Team_B_Score':team_b_score,
                 'Match_Status':match_status, 'Chanel_Streaming':chanel_streaming,
                 'Match_Detail_Link':match_detail_link,'Match_Week_Number':match_week_number}
        
        # Appending the match to match_list
        match_details.append(match)
        
        return(match_details)
        

            

 #### Storing the data in csv file

In [36]:
# Function to store the data scraped in csv file

def store_in_csv(match_details, file_name, path):
    #C:\\Users\\yusuf\\Data Science\\Projects\\Web Scraping\\Yallakor.com"+"\\


    # Creating Keys to use in match dictionary
    keys = ['Date', 'Championship_Title', 'Team_A', 'Team_B', 'Time', 'Team_A_Score', 'Team_B_Score',
            'Match_Status', 'Chanel_Streaming', 'Match_Detail_Link','Match_Week_Number']

    # Storing the championships details in csv file
    with open(path, 'w', newline="") as match_data:
        dict_writer = csv.DictWriter(match_data, fieldnames=keys) 
        dict_writer.writeheader()

        # Looping through championships to store its data
        for match in match_details: 
            dict_writer.writerow(match)

        # Checking the file creation is done
        print(f"Match data saved to file{file_name}")

#### Calling the main function that scrape the website contect

In [37]:
# Function to do the steps of scraping
def scraping(page):
    
    src = page.content
    soup =  BeautifulSoup(src, "lxml")
    
    # List to store matches # Creating list to store match details on it
    match_details = []
    
    championships = soup.find_all("div", {'class':'matchCard'})
    
    # Test championships 
    #print(championships)
    
    
    # Calling the function that gets match info    
    match_details_scraped = get_match_info(championships)
    
    
    file_name = f"Match_Data_Scraped_{new_date_string}.csv"
    path = "C:/Users/yusuf/Data Science/Projects/Web Scraping/Yallakor.com"+"/"+file_name
    
    
    # Calling the function that stores the data scraped in csv file
    store_in_csv(match_details_scraped, file_name, path)


In [38]:
scraping(page)

Match data saved to fileMatch_Data_Scraped_12-30-2022.csv
